## Welcome to your notebook.


#### Run this cell to connect to your GIS and get started:

In [1]:
from arcgis.gis import GIS
gis = GIS("home")

/opt/conda/lib/python3.11/site-packages/arcgis/gis/__init__.py:731: UserWarning: You are logged on as ceco.ai with an administrator role, proceed with caution.
  warnings.warn(


#### Now you are ready to start!

In [2]:
from arcgis.features import FeatureLayer
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
from itertools import combinations
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import random
import http.client
import json
from datetime import datetime, timedelta, timezone
from arcgis.features import GeoAccessor
from arcgis.geometry import SpatialReference
from arcgis.features import FeatureLayer

In [3]:
#set url
feature_layer_url_weather = "https://services6.arcgis.com/bKFxht3iaJpxZRCM/arcgis/rest/services/Group4_weather/FeatureServer/0"
feature_layer_url_air = "https://services6.arcgis.com/bKFxht3iaJpxZRCM/arcgis/rest/services/Group4_air/FeatureServer/0"
feature_layer_weather = FeatureLayer(feature_layer_url_weather)
feature_layer_air = FeatureLayer(feature_layer_url_air)

In [4]:
# query data and transfer to Pandas DataFrame
weather = feature_layer_weather.query(where="1=1")
air = feature_layer_air.query(where="1=1")
df_weather = weather.sdf
df_air = air.sdf

In [5]:
#clean weather data
double_df = df_weather.drop(columns=['OBJECTID','createdAt','precipType','SHAPE','lat','lng','icon'])
double_df = double_df.fillna(double_df.mean())
df_combined = pd.concat([double_df, df_weather[['createdAt']]], axis=1)

#clean air data
df_air = df_air.drop(columns=['OBJECTID','postalCode','SHAPE'])

#orginal dataframe
df = pd.merge(df_combined, df_air, on='createdAt', how='outer')
df['createdAt'] = pd.to_datetime(df['createdAt'], errors='coerce')
df = df.drop(columns=['lat','lng'])
df = df.drop_duplicates()

In [6]:
#Create LabelEncoder
label_encoder = LabelEncoder()
df['majorPollutant'] = label_encoder.fit_transform(df['majorPollutant'])

#sort
df_sorted = df.sort_values(by='createdAt', ascending=True)

df_sorted = df_sorted.set_index('createdAt')

In [7]:
df = df_sorted
original_columns = df.columns

# Lagged time series window
lag_windows = [1, 3, 12]
moving_average_windows = [3, 12, 24]

# Hysteresis characteristics
for column in original_columns:
    for lag in lag_windows:
        df[f'{column}_lag_{lag}h'] = df[column].shift(lag)

# Moving average characteristics
for column in original_columns:
    for window in moving_average_windows:
        df[f'{column}_ma_{window}h'] = df[column].shift(1).rolling(window=window).mean()

df

,cloudCover,dewPoint,humidity,pressure,precipIntensity,surfacePressure,temperature,visibility,windGust,windSpeed,...,SO2_ma_24h,OZONE_ma_3h,OZONE_ma_12h,OZONE_ma_24h,AQI_ma_3h,AQI_ma_12h,AQI_ma_24h,majorPollutant_ma_3h,majorPollutant_ma_12h,majorPollutant_ma_24h
createdAt,,,,,,,,,,,,,,,,,,,,,
2024-11-11 22:00:00,0.92,17.54,89.0,998.0,0.0,840.0,20.0,10.0,2.01,1.57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-11-11 23:00:00,0.82,17.84,93.0,998.0,0.0,840.0,19.0,10.0,1.8,1.92,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-11-12 00:00:00,0.71,13.09,97.0,999.0,0.0,841.0,14.0,10.0,2.26,2.39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-11-12 01:00:00,0.56,8.0,98.0,1001.0,0.0,841.0,10.0,7.31446,3.16,3.5,...,NaN,21.509667,NaN,NaN,20.000000,NaN,NaN,0.0,NaN,NaN
2024-11-12 02:00:00,0.11,5.88,98.0,1003.0,0.0,841.0,8.0,8.3368,3.13,3.58,...,NaN,20.773000,NaN,NaN,19.333333,NaN,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-03 17:00:00,1.0,23.98,95.0,1018.0,0.03,859.0,25.0,10.0,5.85,4.62,...,0.024417,18.487000,19.665583,18.231000,17.000000,18.416667,17.083333,0.0,0.166667,0.166667
2024-12-03 18:00:00,1.0,25.45,97.0,1018.0,0.27,859.0,26.0,0.30932,6.74,4.71,...,0.024000,18.760000,19.550833,18.325250,17.000000,18.166667,17.166667,0.0,0.083333,0.166667
2024-12-03 19:00:00,1.0,27.73,98.0,1017.0,0.2,859.0,28.0,0.31979,6.75,4.54,...,0.023583,18.921333,19.449250,18.508875,17.333333,18.083333,17.333333,0.0,0.083333,0.166667


In [8]:
fut_df = df
columns_to_drop = [col for col in original_columns if col != 'AQI']
df = df.drop(columns=columns_to_drop)

# Spearman
spearman_corr = df.corr(method='spearman')

threshold = 0.3  
significant_features = spearman_corr['AQI'][abs(spearman_corr['AQI']) > threshold].index.tolist()

df = df.loc[:, significant_features]

df = df.dropna()

In [9]:
# Train model
X = df.drop(columns=['AQI'])
y = df['AQI']

results = []
# First for-loop: Randomly select 10 features
for subset_iteration in range(10):  # Change 10 to the number of random subsets you want to try
    # Randomly select 10 features
    selected_features = random.sample(list(X.columns), 10)
    X_subset = X[selected_features]
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_subset, y, test_size=0.2, random_state=42)
    
    # Standardization
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Define the XGBoost model
    model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
    
    # Define hyperparameter grid for random search
    param_distributions = {
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7, 10],
        'n_estimators': [50, 100, 150],
        'colsample_bytree': [0.6, 0.8, 1.0],  # Fraction of features used per tree
        'subsample': [0.6, 0.8, 1.0]  # Fraction of data used for training each tree
    }
    
    # Randomized search for the best hyperparameters
    randomized_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_distributions,
        n_iter=10,  # Number of parameter settings sampled
        scoring='r2',  # Metric to optimize
        cv=3,  # 3-fold cross-validation
        random_state=42,
        n_jobs=-1
    )
    
    # Fit the randomized search to the training data
    randomized_search.fit(X_train_scaled, y_train)
    
    # Best parameters and model performance
    best_params = randomized_search.best_params_
    best_model = randomized_search.best_estimator_
    train_r2 = best_model.score(X_train_scaled, y_train)
    test_r2 = best_model.score(X_test_scaled, y_test)
    
    # Store results for this subset
    results.append({
        'Subset': subset_iteration + 1,
        'Selected Features': selected_features,
        'Best Parameters': best_params,
        'Train R²': train_r2,
        'Test R²': test_r2
    })

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Display the best performing subset
best_result = results_df.loc[results_df['Test R²'].idxmax()]

In [10]:
feature = best_result['Selected Features']
X = df[feature]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
# Standardization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

params = best_result['Best Parameters']
    
# Define the XGBoost model
model = xgb.XGBRegressor(**params, random_state=42)
    
# Fit the randomized search to the training data
model.fit(X_train_scaled, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=50, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

In [11]:
fut_df = fut_df.iloc[:, :23]
last_time = fut_df.index[-1]
next_time = last_time + pd.Timedelta(hours=1)

new_row = pd.DataFrame({col: [np.nan] for col in fut_df.columns}, index=[next_time])

fut_df = pd.concat([fut_df, new_row])

In [12]:
original_columns = fut_df.columns

# Lagged time series window
lag_windows = [1, 3, 12]
moving_average_windows = [3, 12, 24]

# Hysteresis characteristics
for column in original_columns:
    for lag in lag_windows:
        fut_df[f'{column}_lag_{lag}h'] = fut_df[column].shift(lag)

# Moving average characteristics
for column in original_columns:
    for window in moving_average_windows:
        fut_df[f'{column}_ma_{window}h'] = fut_df[column].shift(1).rolling(window=window).mean()

In [13]:
# predict
real = fut_df[feature].iloc[-1]
input_data = real.values.reshape(1, -1)
result = model.predict(input_data)

In [14]:
real_time = pd.DataFrame({'AQI': [float(result)]})
real_time['lat'] = 57.733
real_time['lng'] = -129.7677

# tranfer DataFrame to Spatial DataFrame
spatial_df = pd.DataFrame.spatial.from_xy(real_time, 'lng', 'lat',sr=SpatialReference(4326))
feature_set = spatial_df.spatial.to_featureset()

In [15]:
def upload_data_to_feature_layer(data, layer_url):
    #Check feature layer url
    layer = FeatureLayer(feature_layer_url)
     
    # Get existing features
    existing_features = layer.query(where="1=1")
    
    # Delete all existing features
    if len(existing_features.features) > 0:
        layer.delete_features(deletes=','.join([str(feature.attributes['OBJECTID']) for feature in existing_features.features]))
    
    #upload
    layer.edit_features(adds=data)
    
#set url
feature_layer_url = "https://services6.arcgis.com/bKFxht3iaJpxZRCM/arcgis/rest/services/Group4_real_time/FeatureServer/0"

#upload data
upload_data_to_feature_layer(feature_set, feature_layer_url)